In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Lambda, Conv2D
from keras.layers import GlobalAveragePooling2D, Input, Dropout
from keras.layers.convolutional import MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image

from keras.optimizers import Adam, RMSprop, SGD
import urllib
from os.path import join
from sklearn import model_selection
from sklearn import metrics

/home/igor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../input/'
model_path = path + 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
target_size = (224,224)
INPUT_SIZE = 224
BATCH_SIZE = 8
NUM_CLASSES = 2
EPOCHS = 10
SEED = 1998

In [3]:
def preprocess_img(img):
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=(np.float32))
    img = (img - vgg_mean)
    return img[:, ::-1] # BGR

### Criando modelo

In [4]:
def create_vgg16(x=None):
    model = Sequential()
    
    # layer to preprocess images
    model.add(Lambda(preprocess_img, input_shape=(224,224,3), output_shape=(224,224,3)))
    #Conv2D(64, kernel_size=(3,3), input_shape=(224,224,3), activation='relu', padding='same')
    # conv block 1
    model.add(Conv2D(32, kernel_size=(3,3), input_shape=(224,224,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    # conv block 2
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.5))

    # conv block 3
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.5))
    # Fully Connected
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1000, activation='softmax'))
    
    return model

### Preparando Input

In [5]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join(path, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [6]:
labels = pd.read_csv('../input/labels.csv')
submissao = pd.read_csv('../input/sample_submission.csv')

In [8]:
print(labels.shape[0])
print(submissao.shape[0])

10222
10357


In [9]:
selected_breeds = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).index)

In [10]:
labels = labels[labels['breed'].isin(selected_breeds)]

In [11]:
labels['value'] = 0
#labels['rank'] = labels.groupby('breed').rank()['id']

In [12]:
labels_pivot = labels.pivot(index='id', columns='breed', values='value').reset_index().fillna(0)
labels_pivot = labels_pivot.set_index('id')

In [13]:
train_data = []
target = pd.DataFrame(columns=selected_breeds)

In [15]:
array = [0 for x in range(labels_pivot.shape[1])]

In [16]:
for x in labels.values:
    train_data.append(read_img(x[0], 'train', (150,150)))
    target.loc[target.shape[0]] = array  
    target.loc[target.shape[0]-1][str(x[1])] = 1

In [15]:
initial_model = create_vgg16()
#initial_model.load_weights('/home/igor/projetos/tcc/input/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

# Truncate and replace softmax layer for transfer learning
x = Dense(120, activation='softmax')(initial_model.layers[-2].output)
model = Model(initial_model.input, x)


for layer in model.layers:
    layer.trainable = True
    
opt = SGD(lr=10e-5)
model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(train_data), np.array(target),batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/10
10222/10222 [==============================] - 2750s 269ms/step - loss: 16.0051 - acc: 0.0069
Epoch 2/10
 7592/10222 [=====================>........] - ETA: 11:06 - loss: 15.9759 - acc: 0.0088

In [ ]:
submissao 

In [ ]:
#y_pred = model.predict(batch_size=BATCH_SIZE)

In [ ]:
#y_pred = model.predict(batch_size=BATCH_SIZE)

In [ ]:
#y_pred[0]